In [6]:
import glob
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2

In [8]:
from pandarallel import pandarallel
pandarallel.initialize(8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [49]:
def img2jpeg(p):
    try:
        img = Image.open(p)
        img.verify()
        if img.format!='JPEG':
            img.save(p)
            print(img.format)
    except:
        try:
            p = str(p)
            img = cv2.imread(p)
            cv2.imwrite(p, img)
        except:
            dirty_list.append(p)

In [50]:
%%time
df_test = df1.drop_duplicates('img_sp')
dirty_list = []
_ = df_test['img_sp'].parallel_apply(lambda x: img2jpeg(x))
# _ = df_test['img_sp'].apply(lambda x: img2jpeg(x))

CPU times: user 52.2 ms, sys: 69.2 ms, total: 121 ms
Wall time: 497 ms


In [53]:
%%time
def cv2_isNone(p):
    img = cv2.imread(p)
    flag = False if img is None else True
    return flag

df_test = df1.drop_duplicates('img_sp')
pandarallel.initialize(8)
df_test = df1.drop_duplicates('img_sp')
dirty_list = df_test['img_sp'].parallel_apply(lambda x: cv2_isNone(x))
df_test['flag'] = dirty_list
dirty_df = df_test[~df_test['flag']]
df_test.value_counts('flag')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 123 ms, sys: 57.4 ms, total: 181 ms
Wall time: 1min 30s


<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


flag
True    45529
dtype: int64

In [47]:
csv_p = '/home/chenyi/workspace/dataset/labels/labels6/fashionpedia/csv/fashionpedia6_l1_cat.csv'
df1 = pd.read_csv(csv_p)

,source,img_sp,width,height,bx1,bx2,bx3,bx4,label_model,mode
0,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,681,1024,0.551395,0.898926,0.077827,0.124023,0,train
1,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,681,1024,0.477974,0.869629,0.063142,0.038086,0,train
2,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,682,1024,0.565249,0.833008,0.095308,0.130859,0,val
3,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,682,1024,0.489003,0.843750,0.086510,0.138672,0,val
4,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,683,1024,0.445827,0.887695,0.185944,0.099609,0,val
...,...,...,...,...,...,...,...,...,...,...
125604,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,682,1024,0.559384,0.616699,0.200880,0.415039,5,val
125605,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,579,1024,0.479275,0.419922,0.502591,0.386719,5,train
125606,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,682,1024,0.409091,0.511719,0.275660,0.386719,5,train
125607,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,682,1024,0.496334,0.480957,0.332845,0.385742,5,train


In [13]:
dup_df = pd.read_csv('./data/drop_dup.csv')
# df1.to_csv(csv_p, index=False)

In [42]:
df1['img_sp'] = df1['img_sp'].apply(lambda x: Path(x))

In [35]:
img_df = pd.DataFrame()
img_df['img_sp'] = list(Path('/home/chenyi/workspace/dataset/data/haowei34k/images/').rglob('*.jpg'))
len(img_df)

30893

In [43]:
cross_df = df1[df1['img_sp'].isin(img_df['img_sp'])]
len(df1), len(cross_df)

(143966, 130663)

In [45]:
len(cross_df.drop_duplicates('img_sp'))

30284

In [56]:
cross_df = df1
fname = 'fashionpedia'
root_tran = Path(f'/home/chenyi/workspace/dataset/trainsets/single/{fname}')
if not root_tran.is_dir(): root_tran.mkdir(parents=True)

train_df = cross_df[cross_df['mode']=='train']
val_df = cross_df[cross_df['mode']=='val']
train_df.to_csv(root_tran /f'{fname}_label6_train.csv', index=False)
val_df.to_csv(root_tran /f'{fname}_label6_val.csv', index=False)
len(train_df), len(val_df)

(100455, 25154)

In [46]:
# cross_df.to_csv('/home/chenyi/workspace/dataset/data/haowei34k/annotations/csv/bboxes_yolo_clear30k', index=False)

### CHECK

In [ ]:
len(list(Path('/home/chenyi/workspace/dataset/data/haowei34k/images/').rglob('*.jpg')))

### 删除多余数据

In [17]:
dirty_df

,source,img_sp,width,height,bx1,bx2,bx3,bx4,label_model,mode,flag
55590,haowei34k,/home/chenyi/workspace/dataset/data/haowei34k/...,300,400,0.540000,0.310000,0.280000,0.320000,2,train,False
124142,haowei34k,/home/chenyi/workspace/dataset/data/haowei34k/...,400,600,0.487500,0.525833,0.230000,0.661667,5,train,False
125361,haowei34k,/home/chenyi/workspace/dataset/data/haowei34k/...,640,960,0.439844,0.323437,0.429688,0.263542,2,train,False


In [23]:
!ls /nas/chenyi/datasets_cls/img_remove/

In [31]:
from tqdm import tqdm
import shutil

del_df = drop_dup_df.drop_duplicates('img_sp')
troot = Path('/nas/chenyi/datasets_cls/img_remove')
if not troot.is_dir(): troot.mkdir(parents=True)
for p in tqdm(del_df['img_sp'].values):
    p_lib = Path(p)
    if p_lib.is_file():
        tp = troot /p_lib.name
        shutil.move(p_lib, tp)

100%|███████████████████████████████████████| 3127/3127 [00:03<00:00, 784.05it/s]
